<a href="https://colab.research.google.com/github/JonahMoua/cm1/blob/master/CM1_DevQuickTest_Question.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Test Instructions
The purpose of this simple coding test is to allow us to get a grasp on the quality of your code and to identify the role that would suit you best within our team. You don't NEED to be able to answer all of the items, but the ones that you do answer need to be correct. Feel free to use Google or any other tools that you prefer to complete these tasks.

Install any needed third-party libraries below this block. Please install the minimum amount of libraries you need.

!pip install whatever_you_need

In [ ]:
pip install -U discord.py

In [ ]:
pip install langchain

In [ ]:
pip install openai

In [19]:
pip install asyncio

In [29]:
from google.colab import files

data = files.upload()

Saving message_data.csv to message_data (2).csv


### Task 1 (Data manipulation / Visualization)
Download the file: https://drive.google.com/file/d/1R_M3xI3b_BqAN_xWLqJDneg273X_sok5/view?usp=sharing

In `message_data.csv` there's two columns, one corresponds to the unique id of someone sending a message and the other one to when they sent it.

Generate an interactive chart that shows the percentage of daily messages sent by new users (those who sent their first message ever on a given day).

In [30]:
import plotly.express as px
import pandas as pd

df = pd.read_csv("message_data.csv")

df['message_time'] = pd.to_datetime(df['message_time'])

daily_counts = df.groupby(df['message_time'].dt.date)['author_id'].nunique().reset_index()

daily_counts['Cumulative_Users'] = daily_counts['author_id'].cumsum()

daily_counts['Percentage_New_Users'] = daily_counts['author_id'] / daily_counts['Cumulative_Users'] * 100

fig = px.line(daily_counts, x='message_time', y='Percentage_New_Users', title='Percentage of Daily Messages Sent by New Users')

fig.show()


### Task 2 (Django)
**To take the django portion of this test go to: https://github.com/llu13701/cm1_python_test and follow the instructions.**

### Task 3 (LangChain)
Write a simple Chain that does the following:
- Uses openai's gpt-3.5-turbo model.
- Remembers the last 2 message exchanges, as well as the system message.
- Greets people and tells a joke about their name.

In [31]:
from langchain.llms import OpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain


API = "OPENAI_API_KEY"
llm = OpenAI(model_name="gpt-3.5-turbo", openai_api_key=API)


prompt = PromptTemplate(
    input_variables=["name"],
    template="Greet {name} and tell them a joke using their name.",
)
chain = LLMChain(llm=llm, prompt=prompt)

second_prompt = PromptTemplate(
    input_variables=["name"],
    template="Give a different joke that uses their name {name}",
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

overall_chain = SimpleSequentialChain(chains=[chain, chain_two], verbose=True)

explanation = overall_chain.run("Elisa")
print(explanation)


/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:173: UserWarning:

You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`

/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:751: UserWarning:

You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`





> Entering new SimpleSequentialChain chain...
Hello Elisa! How are you today? Here's a joke just for you:

Why did Elisa bring a ladder to the bar?
Because she heard the drinks were on the house! ☺
Hey Elisa! How about another joke just for you:

Why did Elisa go to the bakery?
Because she kneaded a break from all the work! 😄

> Finished chain.
Hey Elisa! How about another joke just for you:

Why did Elisa go to the bakery?
Because she kneaded a break from all the work! 😄


### Task 4 (Discord API)

Write a very simple discord bot that says "Hello {user_name}!" to every new incoming message in a discord server.


In [32]:
import discord
from discord.ext import commands
import threading
import asyncio

intents = discord.Intents.default()
intents.message_content = True

bot = commands.Bot(command_prefix='!', intents=intents)


@bot.event
async def on_ready():
    print(f'Logged in as {bot.user.name}')


@bot.event
async def on_message(message):
    if message.author.bot:
        return

    await message.channel.send(f'Hello {message.author.name}!')

    await bot.process_commands(message)


async def run_bot():
    await bot.start('YOURE_API_KEY')


def run_bot_in_thread():
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(run_bot())


thread = threading.Thread(target=run_bot_in_thread)
thread.start()

### Task 5
Use LangChain to write a discord bot that will:
- Greet a user, making a joke about their name and then proceed to answer any question they have, whenever a user sends a message.
- Use async with langchain chain calls.
